# Building with LangChain

In [3]:
import os
import sys
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

load_dotenv()  # Load API keys and other sensitive data from the .env file

True

In [4]:
api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

# LLM Chain

In [5]:
llm = ChatOpenAI(openai_api_key=api_key)  # Create the ChatGPT instance

In [6]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing by providing automated testing tools and frameworks that can be used to test various aspects of software, such as functionality, performance, and security. These tools can help streamline the testing process, save time, and improve the overall quality of the software being developed. Langsmith can also provide testing services, such as test case creation, execution, and reporting, to ensure that the software meets the required standards and specifications. Additionally, Langsmith can help with test automation, continuous integration, and continuous deployment to ensure that testing is integrated seamlessly into the software development process.')

In [7]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [8]:
chain = prompt | llm 

In [9]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='Langsmith can be a valuable tool for testing in various ways. Here are some ways Langsmith can help with testing:\n\n1. Automated Testing: Langsmith can be used to create automated tests for different components of your software. By generating test data and test cases, Langsmith can help in automating the testing process, saving time and effort.\n\n2. Test Data Generation: Langsmith can generate diverse and realistic test data for various scenarios, helping in thorough testing of the software. This can ensure better test coverage and help in identifying potential issues early in the development cycle.\n\n3. Performance Testing: Langsmith can be used to generate load and stress testing data to evaluate the performance of your software under different conditions. This can help in identifying performance bottlenecks and optimizing the software for better performance.\n\n4. API Testing: Langsmith can generate API calls and test data for API testing. This can help in vali

In [10]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [11]:
chain = prompt | llm | output_parser

In [12]:
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can help with testing in a few different ways:\n\n1. Automated Testing: Langsmith can be used to automate testing processes by generating test cases, executing tests, and comparing expected versus actual results. This can help in identifying bugs and ensuring software quality.\n\n2. Test Data Generation: Langsmith can be used to generate test data for various scenarios, helping in thorough testing of the software under different conditions.\n\n3. Code Coverage Analysis: Langsmith can be used to analyze code coverage during testing, ensuring that all parts of the code are exercised by the tests.\n\n4. Performance Testing: Langsmith can be used to simulate different load conditions and stress test the software to identify performance bottlenecks.\n\nOverall, Langsmith can be a valuable tool in the testing process, helping to streamline testing efforts and improve the quality of the software being tested.'

# Retrieval Chain

In [13]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")

docs = loader.load()

In [14]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=api_key)

In [15]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [17]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing you to visualize test results.'

In [18]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [19]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith can help with testing by providing the ability to debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework. It also seamlessly integrates with LangChain, the open source framework for building with LLMs. Additionally, LangSmith offers tracing and evaluation capabilities to assist with testing applications.


# Conversation Retrieval Chain

In [20]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [21]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith supports at each stage of the LLM application lifecycle.Setup: Learn 

In [22]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [23]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following s

# Agent

We've so far create examples of chains - where each step is known ahead of time. The final thing we will create is an agent - where the LLM decides what steps to take.

NOTE: for this example we will only show how to create an agent using OpenAI models, as local models are not reliable enough yet.

One of the first things to do when building an agent is to decide what tools it should have access to. For this example, we will give the agent access to two tools:

The retriever we just created. This will let it easily answer questions about LangSmith
A search tool. This will let it easily answer questions that require up to date information.
First, let's set up a tool for the retriever we just created:

In [24]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [25]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [26]:
tools = [retriever_tool, search]

In [27]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [28]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


LangSmith | 🦜️🛠️ LangSmith





Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction​LangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start​Tracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps​Check out the following sections to learn more about LangSmith:User Guide: Learn about the workfl

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It can help with testing by allowing you to debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework. LangSmith seamlessly integrates with LangChain, which is an open-source framework for building with LLMs. \n\nYou can use LangSmith for tracing, evaluation, and monitoring your LLM applications. It supports various workflows at each stage of the LLM application lifecycle. Additionally, LangSmith offers capabilities for tracing and evaluation, which are essential for testing and optimizing LLM applications.\n\nIf you want to learn more about how LangSmith can specifically assist with testing, you can explore the User Guide, Setup, Pricing, Self-Hosting, Tracing, Evaluation, and Prompt Hub sections on the LangSmith platform.'}

In [29]:
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.accuweather.com/en/us/san-francisco/94103/current-weather/347629', 'content': 'Get the latest weather conditions and forecast for San Francisco, CA, including temperature, precipitation, wind, humidity, and UV index. Be aware of flood, wind, and surf advisories in effect for the next few days.'}, {'url': 'https://weather.com/weather/tenday/l/San Francisco CA USCA0987:1:US', 'content': "Comfy & Cozy\nThat's Not What Was Expected\nOutside\n'No-Name Storms' In Florida\nGifts From On High\nWhat To Do For Wheezing\nSurviving The Season\nStay Safe\nAir Quality Index\nAir quality is considered satisfactory, and air pollution poses little or no risk.\n Health & Activities\nSeasonal Allergies and Pollen Count Forecast\nNo pollen detected in your area\nCold & Flu Forecast\nFlu risk is low in your area\nWe recognize our responsibility to use data and

{'input': 'what is the weather in SF?',
 'output': 'I found some sources where you can check the weather in San Francisco:\n1. [AccuWeather - San Francisco Current Weather](https://www.accuweather.com/en/us/san-francisco/94103/current-weather/347629): Get the latest weather conditions and forecast for San Francisco, CA, including temperature, precipitation, wind, humidity, and UV index.\n2. [Weather.com - 10 Day Weather Forecast for San Francisco, CA](https://weather.com/weather/tenday/l/San Francisco CA USCA0987:1:US): Check the 10-day weather forecast for San Francisco, including details for today and the upcoming days.\n3. [AccuWeather - San Francisco Weather Forecast](https://www.accuweather.com/en/us/san-francisco/94103/weather-forecast/347629): View the current weather, radar, and air quality for San Francisco, CA, along with the forecast for the next week.\n4. [Weather Underground - San Francisco Weather Information](https://www.wunderground.com/forecast/us/ca/san-francisco): Ge

In [30]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith LLM application testing services'}`


LangSmith | 🦜️🛠️ LangSmith





Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction​LangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start​Tracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps​Check out the following sections to learn more about LangSmith:User Guide: Learn about the

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': "LangSmith is a platform for building production-grade LLM applications. It allows you to debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework. LangSmith seamlessly integrates with LangChain, the open-source framework for building with LLMs. You can learn more about LangSmith's workflows, setup, pricing, self-hosting options, tracing capabilities, evaluation capabilities, and more by referring to the LangSmith documentation and resources provided by LangChain."}

# Serving with LangServe
Now that we've built an application, we need to serve it. That's where LangServe comes in. LangServe helps developers deploy LangChain chains as a REST API. You do not need to use LangServe to use LangChain, but in this guide we'll show how you can deploy your app with LangServe.

While the first part of this guide was intended to be run in a Jupyter Notebook, we will now move out of that. We will be creating a Python file and then interacting with it from the command line.

Install with:

In [33]:
#!/usr/bin/env python
from typing import List

from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.messages import BaseMessage
from langserve import add_routes

# 1. Load Retriever
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings()
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

# 2. Create Tools
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)
search = TavilySearchResults()
tools = [retriever_tool, search]


# 3. Create Agent
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


# 4. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 5. Adding chain route

# We need to add these input/output schemas because the current AgentExecutor
# is lacking in schemas.

class Input(BaseModel):
    input: str
    chat_history: List[BaseMessage] = Field(
        ...,
        extra={"widget": {"type": "chat", "input": "location"}},
    )


class Output(BaseModel):
    output: str

add_routes(
    app,
    agent_executor.with_types(input_type=Input, output_type=Output),
    path="/agent",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)

RuntimeError: asyncio.run() cannot be called from a running event loop

we should see our chain being served at localhost:8000.

# Playground
Every LangServe service comes with a simple built-in UI for configuring and invoking the application with 
streaming output and visibility into intermediate steps. Head to http://localhost:8000/agent/playground/ 
to try it out! Pass in the same question as before - "how can langsmith help with testing?" - and it 
should respond same as before.

# Client
Now let's set up a client for programmatically interacting with our service. We can easily do this 
with the [langserve.RemoteRunnable](/docs/langserve#client). Using this, we can interact with the 
served chain as if it were running client-side.